In [2]:
"""
간단한 TCP/IP 스모크 진단 도구
- ping (ICMP)으로 인터넷 계층 체크
- TCP 연결 시도(포트)로 전송 계층 체크
- HTTP GET(응용)으로 애플리케이션 체크
"""
import subprocess
import re
import socket
import urllib.request
import urllib.error


class NetworkDiagnostics:
    def ping_test(self, host, count=4, timeout=10):
        """ICMP ping 검사 (Linux 기준).
        반환: {'status':'success'|'failed'|'error', 'avg_ms': float or None, 'raw': str}
        """
        try:
            proc = subprocess.run(
                ["ping", "-c", str(count), host],
                capture_output=True,
                text=True,
                timeout=timeout,
            )
            out = proc.stdout + proc.stderr
            if proc.returncode == 0:
                # Linux ping: rtt min/avg/max/mdev = 0.042/0.042/0.042/0.000 ms
                m = re.search(r"rtt [\w/]+ = [\d.]+/([\d.]+)/[\d.]+/[\d.]+ ms", out)
                if not m:
                    # 다른 ping 변형(예: macOS) min/avg/max/stddev = …
                    m = re.search(r"round-trip.*= [\d.]+/([\d.]+)/", out)
                avg = float(m.group(1)) if m else None
                return {"status": "success", "avg_ms": avg, "raw": out}
            else:
                return {"status": "failed", "avg_ms": None, "raw": out}
        except Exception as e:
            return {"status": "error", "message": str(e)}

    def tcp_connectivity_test(self, host, port, timeout=5):
        """TCP 포트 연결성 테스트. 반환: status, errno"""
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(timeout)
            err = sock.connect_ex((host, port))
            sock.close()
            return {"status": "success" if err == 0 else "failed", "errno": err}
        except Exception as e:
            return {"status": "error", "message": str(e)}

    def http_test(self, url, timeout=5):
        """간단한 HTTP GET 검사(앱 레벨)."""
        try:
            req = urllib.request.Request(url, headers={"User-Agent": "diag/1.0"})
            with urllib.request.urlopen(req, timeout=timeout) as resp:
                code = resp.getcode()
                length = resp.getheader("Content-Length")
                return {
                    "status": "success" if 200 <= code < 400 else "failed",
                    "status_code": code,
                    "content_length": length,
                }
        except urllib.error.HTTPError as he:
            return {"status": "failed", "status_code": he.code, "message": str(he)}
        except Exception as e:
            return {"status": "error", "message": repr(e)}


if __name__ == "__main__":
    nd = NetworkDiagnostics()
    target = "example.com"
    print("Ping:", nd.ping_test(target))
    print("TCP 80:", nd.tcp_connectivity_test(target, 80))
    print("HTTP:", nd.http_test("https://" + target))

Ping: {'status': 'failed', 'avg_ms': None, 'raw': '액세스가 거부되었습니다. -c 옵션을 사용하려면 관리 권한이 필요합니다.\n'}
TCP 80: {'status': 'success', 'errno': 0}
HTTP: {'status': 'success', 'status_code': 200, 'content_length': '1256'}


In [3]:
import ipaddress, math


def allocate_vlsm(base_net_cidr, requirements):
    base = ipaddress.IPv4Network(base_net_cidr)
    # 정렬: 큰 호스트 요구 먼저
    reqs = sorted(requirements, reverse=True)
    result = []
    cursor = int(base.network_address)
    end = int(base.broadcast_address) + 1

    for hosts in reqs:
        needed = hosts + 2
        size = 1 << math.ceil(math.log2(needed))
        prefix = 32 - int(math.log2(size))
        # find aligned network
        # CIDR requires network address aligned to block size
        while True:
            try_net = ipaddress.IPv4Network((ipaddress.IPv4Address(cursor), prefix))
            if int(try_net.network_address) != cursor:
                # align to next boundary
                cursor = int(try_net.network_address)
            if int(try_net.broadcast_address) < end:
                result.append(
                    {
                        "network": str(try_net),
                        "usable": (
                            str(list(try_net.hosts())[0]),
                            str(list(try_net.hosts())[-1]),
                        ),
                        "hosts": hosts,
                    }
                )
                cursor = int(try_net.broadcast_address) + 1
                break
            else:
                raise ValueError("Not enough space")
    return result


# 사용 예
print(allocate_vlsm("192.168.1.0/24", [50, 25, 10, 5]))


[{'network': '192.168.1.0/26', 'usable': ('192.168.1.1', '192.168.1.62'), 'hosts': 50}, {'network': '192.168.1.64/27', 'usable': ('192.168.1.65', '192.168.1.94'), 'hosts': 25}, {'network': '192.168.1.96/28', 'usable': ('192.168.1.97', '192.168.1.110'), 'hosts': 10}, {'network': '192.168.1.112/29', 'usable': ('192.168.1.113', '192.168.1.118'), 'hosts': 5}]


In [4]:
class PATSim:
    def __init__(self, public_ip, port_range=(10000, 11000)):
        self.pub = public_ip
        self.minp, self.maxp = port_range
        self.next_port = self.minp
        self.map = {}  # (priv_ip, priv_port) -> external_port
        self.rev = {}  # external_port -> (priv_ip, priv_port)

    def allocate(self, priv_ip, priv_port):
        key = (priv_ip, priv_port)
        if key in self.map:
            return f"{self.pub}:{self.map[key]}"
        # find free port
        while self.next_port <= self.maxp and self.next_port in self.rev:
            self.next_port += 1
        if self.next_port > self.maxp:
            raise RuntimeError("Port pool exhausted")
        port = self.next_port
        self.map[key] = port
        self.rev[port] = key
        self.next_port += 1
        return f"{self.pub}:{port}"


# 사용 예
pat = PATSim("203.0.113.5")
print(pat.allocate("192.168.0.10", 54321))
print(pat.allocate("192.168.0.11", 54321))


203.0.113.5:10000
203.0.113.5:10001


In [6]:
from netaddr import IPNetwork

# /24 대역을 4개의 /26 서브넷으로 분할
for subnet in IPNetwork("192.168.1.0/24").subnet(26):
    print(subnet)  # 실무 서브넷 분배 결과

192.168.1.0/26
192.168.1.64/26
192.168.1.128/26
192.168.1.192/26


In [7]:
from netaddr import IPNetwork

subnet = IPNetwork("192.168.1.0/26")
print("Network:", subnet.network)
print("Broadcast:", subnet.broadcast)
print("Host Address Range:", [str(ip) for ip in subnet.iter_hosts()])

Network: 192.168.1.0
Broadcast: 192.168.1.63
Host Address Range: ['192.168.1.1', '192.168.1.2', '192.168.1.3', '192.168.1.4', '192.168.1.5', '192.168.1.6', '192.168.1.7', '192.168.1.8', '192.168.1.9', '192.168.1.10', '192.168.1.11', '192.168.1.12', '192.168.1.13', '192.168.1.14', '192.168.1.15', '192.168.1.16', '192.168.1.17', '192.168.1.18', '192.168.1.19', '192.168.1.20', '192.168.1.21', '192.168.1.22', '192.168.1.23', '192.168.1.24', '192.168.1.25', '192.168.1.26', '192.168.1.27', '192.168.1.28', '192.168.1.29', '192.168.1.30', '192.168.1.31', '192.168.1.32', '192.168.1.33', '192.168.1.34', '192.168.1.35', '192.168.1.36', '192.168.1.37', '192.168.1.38', '192.168.1.39', '192.168.1.40', '192.168.1.41', '192.168.1.42', '192.168.1.43', '192.168.1.44', '192.168.1.45', '192.168.1.46', '192.168.1.47', '192.168.1.48', '192.168.1.49', '192.168.1.50', '192.168.1.51', '192.168.1.52', '192.168.1.53', '192.168.1.54', '192.168.1.55', '192.168.1.56', '192.168.1.57', '192.168.1.58', '192.168.1.59',